<a href="https://colab.research.google.com/github/stevejj4/Insurance-data-lifecycle/blob/main/Policies_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install google-cloud-bigquery
!pip install pyspark==3.1.2

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
from pyspark.sql import SparkSession


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 13.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880745 sha256=ed038b30b86863f2a21100cacff0bd409239ce0a5d0eccf1b8e2bf71070002fe
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [3]:
# Authenticating and initializing BigQuery client
project_id = 'river-messenger-430112-e1'
client = bigquery.Client(project=project_id)

In [4]:
# Querying to get interactions table
query_policies = """
SELECT * FROM `river-messenger-430112-e1.Insurance_data.policies`;
"""

In [5]:
# Executing the query and converting to a pandas DataFrame
df_policies = client.query(query_policies).to_dataframe()

In [6]:
# Initializing SparkSession
spark = SparkSession.builder \
    .appName('BigQuerySparkApp') \
    .getOrCreate()

# Converting pandas DataFrame to Spark DataFrame
# Using iterrows() instead of iteritems() to iterate over DataFrame rows
spark_df_interactions = spark.createDataFrame(df_policies.to_dict('records'))

# Show the schema and first few rows
spark_df_interactions.printSchema()
#spark_df_interactions.show(5)

root
 |-- CustomerID: long (nullable = true)
 |-- PolicyEndDate: string (nullable = true)
 |-- PolicyID: long (nullable = true)
 |-- PolicyStartDate: string (nullable = true)
 |-- PolicyType: string (nullable = true)
 |-- PremiumAmount: double (nullable = true)

